# OLCI spectral plotting

This routine shows examples of how to use python netcdf libraries to plot OLCI L2 spectra, either for a single point or averaged over and area.

    Version: 1.1
    Author: B loveday, PML

Import libraries

In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import os, sys, shutil
import fnmatch
import datetime
import logging

Turn off automatic plotting

In [2]:
plt.ioff()

Here we define a function to calculate spherical distance between points so that we can define polygons

In [3]:
# function
def spheric_dist(lat1,lat2,lon1,lon2,mode="global"):

    '''
      function dist=spheric_dist(lat1,lat2,lon1,lon2)
      compute distances for a simple spheric earth

      input:
      lat1 : latitude of first point (matrix or point)
      lon1 : longitude of first point (matrix or point)
      lat2 : latitude of second point (matrix or point)
      lon2 : longitude of second point (matrix or point)

      output:
      dist : distance from first point to second point (matrix)
    '''

    R = 6367442.76
    # Determine proper longitudinal shift.
    l = np.abs(lon2-lon1)
    try:
        l[l >= 180] = 360 - l[l >= 180]
    except:
        pass
    # Convert Decimal degrees to radians.
    deg2rad = np.pi/180
    phi1    = (90-lat1)*deg2rad
    phi2    = (90-lat2)*deg2rad
    theta1  = lon1*deg2rad
    theta2  = lon2*deg2rad
 
    lat1    = lat1*deg2rad
    lat2    = lat2*deg2rad
    l       = l*deg2rad

    if mode=="global":
        # Compute the distances: new
        cos     = (np.sin(phi1)*np.sin(phi2)*np.cos(theta1 - theta2) + 
                   np.cos(phi1)*np.cos(phi2))
        arc     = np.arccos( cos )
        dist    = R*arc
    elif mode=="regional":
        # Compute the distances: 1 old, deprecated ROMS version - unsuitable for global
        dist    = R*np.arcsin(np.sqrt(((np.sin(l)*np.cos(lat2))**2) + (((np.sin(lat2)*np.cos(lat1)) - \
                  (np.sin(lat1)*np.cos(lat2)*np.cos(l)))**2)))
    elif mode=="local":
        #uses approx for now: 
        x = [lon2-lon1] * np.cos(0.5*[lat2+lat1])
        y = lat2-lat1
        dist = R*[x*x+y*y]^0.5
    else:
        print("incorrect mode")

    return dist

Now we define our default parameters

In [4]:
#-default parameters------------------------------------------------------------
DEFAULT_ROOT_DIR    = os.getcwd()
DEFAULT_LOG_PATH    = os.getcwd()
DEFAULT_FILE_FILTER = '*reflectance*.nc'

And any arguments, should we wish to adapt this for batch mode

In [5]:
#-args--------------------------------------------------------------------------
#parser = argparse.ArgumentParser()
#args = parser.parse_args()

And here we enter the programme proper, setting up our preliminaries and logfile.

In [6]:
#-main-------------------------------------------------------------------------
if __name__ == '__main__':
    # preliminary stuff
    logfile = os.path.join(DEFAULT_LOG_PATH,"OLCI_spectral_plot_"+datetime.datetime.now().strftime('%Y%m%d_%H%M')+".log")
    # we define a verbose flag to control how much info we want to see. It can also be useful to define a debug flag
    # for even more information.
    verbose = False
    
    # set file logger
    try:
        if os.path.exists(logfile):
            os.remove(logfile)
        print("logging to: "+logfile)
        logging.basicConfig(filename=logfile,level=logging.DEBUG)
    except:
        print("Failed to set logger")

logging to: /shared/OLCI/OLCI_spectral_plot_20180409_1319.log


Next we define the a box to average our spectra over. If we set the lon values equal, and lat values equal, then we take the spectra from the nearest single point.

In [7]:
    # makes box average. Will choose nearest point if latmin == latmax and lonmin == lonmax
    lonmin = 70.0
    lonmax = 70.0
    latmin = 14.0
    latmax = 14.0

    nearest_flag = False
    if lonmin == lonmax and latmin == latmax:
        print('Using nearest point')
        nearest_flag = True
    else:
        print('Making box average')

Using nearest point


Now we load our latitude and longitude fields so that we can extract the indices for the box (or point) we want to get the spectra for.

In [8]:
    # get the paths and coordinates files    
    input_root    = os.path.join(os.getcwd(),'OLCI_test_data')
    input_path    = "S3A_OL_2_WFR____20171226T045629_20171226T045929_20171227T105453_0179_026_076_2700_MAR_O_NT_002.SEN3"
    file_name_geo = "geo_coordinates.nc"
    # NOTE THAT I HAVE DONE A 'TIE-POINT EXPANSION' HERE TO GET FULL LAT/LON VARIABLES. THIS CAN BE DONE IN SNAP.
    GEO_file      = nc.Dataset(os.path.join(input_root,input_path,file_name_geo), 'r')
    LAT           = GEO_file.variables['latitude'][:]
    LON           = GEO_file.variables['longitude'][:]
    GEO_file.close()

    dist_i1 = spheric_dist(latmin,LAT,lonmin,LON)
    #J is the X-coord
    I1,J1   = np.where(dist_i1 == np.nanmin(dist_i1))

    if nearest_flag:
        I1 = I1[0]
        J1 = J1[0]
        I2 = I1+1
        J2 = J1+1
    else:
        dist_i2 = spheric_dist(latmax,LAT,lonmax,LON)
        I2, J2   = np.where(dist_i2 == np.nanmin(dist_i2))
        I1 = I1[0]
        J1 = J1[0]
        I2 = I2[0]
        J2 = J2[0]
    
    # re-arrange coordinates so that we count upwards...
    if J2 < J1:
        J1f = J2
        J2f = J1
    else:
        J1f = J1
        J2f = J2
    
    if I2 < I1:
        I1f = I2
        I2f = I1
    else:
        I1f = I1
        I2f = I2

The values for the wavelengths of each channel are stored in the xml manifest files, so lets read through that line by line and extract them, as well as the bandwidths.

In [9]:
    # get wavelengths from xml file
    bands              = []
    wavelengths        = []
    bandwidths         = []    
    xml_file = os.path.join(input_root,input_path,'xfdumanifest.xml')
    with open(xml_file, 'r') as input_file:
        for line in input_file:
            if "<sentinel3:band name=" in line:
                bands.append(line.replace('<sentinel3:band name="',"").replace('">','').replace(' ','').replace('\n',''))
            if "<sentinel3:centralWavelength>" in line:
                wavelengths.append(float(line.replace('<sentinel3:centralWavelength>','').replace('</sentinel3:centralWavelength>','')))
            if "<sentinel3:bandwidth>" in line:
                bandwidths.append(float(line.replace('<sentinel3:bandwidth>','').replace('</sentinel3:bandwidth>','')))

Now lets loop through all of the reflectances, one-by-one and pull out the value over the boox or point for each field, storing them in a list.

In [10]:
    # -get the files by band name-------------------------------------------------------------
    nc_files=[]
    for root, _, filenames in os.walk(os.path.join(input_root,input_path)):
        for filename in fnmatch.filter(filenames, DEFAULT_FILE_FILTER):
            nc_files.append(os.path.join(root, filename))
            if verbose:
                print('Found: '+filename)
            logging.info('Found: '+os.path.join(root, filename))

    # get the reflectances
    reflectances       = []
    reflectance_errors = []
    variability        = []
    for nc_file in sorted(nc_files):
        varname  = os.path.basename(nc_file).split('.')[0]
        nc_fid   = nc.Dataset(nc_file, 'r')
        reflectances.append(np.nanmean(nc_fid.variables[varname][I1f:I2f,J1f:J2f]))
        variability.append(np.nanstd(nc_fid.variables[varname][I1f:I2f,J1f:J2f]))
        reflectance_errors.append(np.nanmean(nc_fid.variables[varname+'_err'][I1f:I2f,J1f:J2f]))
        nc_fid.close()
        
    tops = [x + y for x, y in zip(reflectances, variability)]
    bottoms = [x - y for x, y in zip(reflectances, variability)]

Finally, we plot the spectra and save.

In [11]:
    # print the spectra
    fig1 = plt.figure(figsize=(10, 10), dpi=300)
    plt.errorbar(wavelengths, reflectances, xerr=bandwidths, yerr=reflectance_errors,color='k',linewidth=1.0)
    plt.plot(wavelengths,tops,'r--',linewidth=1.0)
    plt.plot(wavelengths,bottoms,'r--',linewidth=1.0)
    plt.fill_between(wavelengths,tops,bottoms,color='r',alpha=0.25)
    plt.xlabel('Wavelength [nm]')
    plt.ylabel('Reflectance')
    plt.savefig('OLCI_spectra_demo.png',bbox_inches='tight')
    plt.close(fig1)